In [1]:
#via_annotations_file = "/media/gregl/Big_Datasets/Grey_Seals/Level_3_1000s/MG2016/MG2016_complete_1st_draft.csv"
tiling_scheme_file = "D:/FCAT/tiles2/tiling_scheme.json" # the tiling scheme JSON file generated during the tiling script
output_dir = "D:/FCAT/annotations" # folder where any outputs will be dumped
input_csv = "D:/FCAT/Orthomosaic_1_training_data_v2.csv" # csv of points representing focal objects, here palm trees
class_colname = 'Especie' # column where the object "class" is located
object_size = 12 # estimated size of our focal object, here a palm tree, given in meters

from pprint import pprint
from shapely.geometry import mapping
import geopandas as gpd, rasterio, os

In [2]:
# this function imports necessary metadata from the tiling scheme file generated during the earlier tiling script
def import_tiling_scheme(tiling_scheme_file):
    import json
    from affine import Affine
    
    with open(tiling_scheme_file) as f:
        tiling_scheme = json.load(f)
    gt = tiling_scheme["transform"]
    geotransform = (gt[2], gt[0], gt[1], gt[5], gt[3], gt[4])
    geotransform = Affine.from_gdal(*geotransform)
    tiling_scheme["transform"] = geotransform
    return tiling_scheme

tiling_scheme = import_tiling_scheme(tiling_scheme_file)

In [3]:
### This section converts the points from the CSV file to boxes
### not necessary if your annotations are already in boxes

# reading in the orthomosaic, for spatial reference (CRS)
# and the point file, as a csv with X and Y coordinate columns for lat/lon
pts = gpd.read_file(input_csv) # read in the CSV file to a geodataframe
gdf=gpd.points_from_xy(pts.POINT_X, pts.POINT_Y)

# we apply a buffer around each point
buffer_dist = object_size/2 # distance from center, in meters
buffer_dist = buffer_dist/111300 # converted to lat/lon degrees

box_list = []
for pt in gdf:
    box= pt.buffer(buffer_dist).envelope
    box_list.append(box)

# then we assemble a new geodataframe with these boxes

d = {'Especie': pts['Especie'], 'geometry': box_list}
boxes = gpd.GeoDataFrame(d, crs=tiling_scheme['spatial_reference'])

# we now have a geodataframe of species-labeled boxes extending 'buffer_dist' around each point
print(boxes[0:5])

             Especie                                           geometry
0    Attalea colenda  POLYGON ((-79.66497 0.36761, -79.66487 0.36761...
1  Oenocarpus bataua  POLYGON ((-79.66444 0.36793, -79.66433 0.36793...
2  Oenocarpus bataua  POLYGON ((-79.66539 0.36803, -79.66528 0.36803...
3  Oenocarpus bataua  POLYGON ((-79.66847 0.36828, -79.66836 0.36828...
4  Oenocarpus bataua  POLYGON ((-79.66939 0.36877, -79.66928 0.36877...


In [4]:
# this function imports necessary metadata from the tiling scheme file generated during the earlier tiling script
def import_tiling_scheme(tiling_scheme_file):
    import json
    from affine import Affine
    
    with open(tiling_scheme_file) as f:
        tiling_scheme = json.load(f)
    gt = tiling_scheme["transform"]
    geotransform = (gt[2], gt[0], gt[1], gt[5], gt[3], gt[4])
    geotransform = Affine.from_gdal(*geotransform)
    tiling_scheme["transform"] = geotransform
    return tiling_scheme

tiling_scheme = import_tiling_scheme(tiling_scheme_file)

In [5]:
# this function converts box coordinates from global coordinates to orthomosaic coordinates
def globalboxes_to_orthoboxes(box_list, tiling_scheme):
    entry_list = []
    for box in boxes.iterrows():
        newbox = {}
        coords = mapping(box[1]['geometry'])['coordinates'][0][0:4]
        newbox['class'] = box[1][class_colname]
        newbox['box'] = []
        for point in coords:
            newbox['box'].append(~tiling_scheme["transform"] * point)
        entry_list.append(newbox)
    return entry_list

orthoboxes = globalboxes_to_orthoboxes(boxes, tiling_scheme)
pprint(orthoboxes[0:5])

[{'box': [(21097.344128131866, 18302.49085371592),
          (21360.015091389418, 18302.49085371592),
          (21360.015091389418, 18041.57306011312),
          (21097.344128131866, 18041.57306011312)],
  'class': 'Attalea colenda'},
 {'box': [(22407.48730790615, 17512.912378759356),
          (22670.158271163702, 17512.912378759356),
          (22670.158271163702, 17251.994585156674),
          (22407.48730790615, 17251.994585156674)],
  'class': 'Oenocarpus bataua'},
 {'box': [(20093.051171183586, 17274.54058739345),
          (20355.722134441137, 17274.54058739345),
          (20355.722134441137, 17013.622793790768),
          (20093.051171183586, 17013.622793790768)],
  'class': 'Oenocarpus bataua'},
 {'box': [(12584.29082006216, 16681.332717996673),
          (12846.961783319712, 16681.332717996673),
          (12846.961783319712, 16420.41492439399),
          (12584.29082006216, 16420.41492439399)],
  'class': 'Oenocarpus bataua'},
 {'box': [(10347.64293706417, 15488.0153938529

In [9]:
# this big ugly section just concerns figuring out which tile(s) a box should be shown in
# it is complicated because boxes can show up in multiple tiles if they straddle the edge
# or if the sit in the overlap region
# but we also want to disregard a box if it exists 90% outside the the tile
# I'm not proud of this function, so if you can do a better job please feel free to!

def assign_tiles(bbox, tiling_scheme):
    tile_height = tiling_scheme["tile_height"] # height of each tile
    tile_width = tiling_scheme["tile_width"] # width of each tile
    tile_overlap = tiling_scheme["tile_overlap"] # overlap between tiles
    img_data = tiling_scheme["tile_pointers"] # top-left pixel location for each image, in orthomosaic coordinates
    x_tile_dist = tile_width - tile_overlap # X-axis stride between tiles
    y_tile_dist = tile_height - tile_overlap # Y-axis stride between tiles

    # set the tile-pointer corners of the box (leftx dividend, topy dividend)
    # and the in-tile coordinates of the box (leftx remainder, topy remainder
    x_coordinates, y_coordinates = zip(*bbox)
    tilepointer_x, intile_lx = divmod(min(x_coordinates), x_tile_dist) # tilepointer tells us the top-left ortho-coordinate of the tile
    tilepointer_y, intile_ty = divmod(min(y_coordinates), y_tile_dist) # intile tells the top-left tile-coordinate of the box

    intile_rx = max(x_coordinates) - (tilepointer_x*x_tile_dist) # rightmost x tile-coordinate of the box
    intile_by = max(y_coordinates) - (tilepointer_y*y_tile_dist) # bottommost y tile-coordinate of the box
    
    box_dimensionx = intile_rx - intile_lx # width of the box in pixels
    box_dimensiony = intile_by - intile_ty # heigh of the box in pixels
    

    tile_pointer = "[{x}, {y}]".format(x=int(tilepointer_x*x_tile_dist), y=int(tilepointer_y*y_tile_dist))
    inv_map = {str(v): k for k, v in img_data['image_locations'].items()}
    # tile_pointer sets the location of the tile, which we use to look up the tile name

    # this variable determines how "clipped" an edge box can be before we throw it away
    disregard_threshold = 0.9
    
    entry = []
    
    # if the rightmost edge of a box is left of the tile's edge + the "disregard threshold"
    # and the bottommost edge is above the tile's edge and the disregard threshold
    if intile_rx < tile_width + disregard_threshold*box_dimensionx and intile_by < tile_height + disregard_threshold*box_dimensiony:
        # we assign the tile name to this box
        try:
            tile_info = inv_map[tile_pointer]
            entry.append(tile_info)
        except:
            print("first attempted tile does not exist (will try an adjacent tile)")
    # use remainder to determine whether a detection occurs in overlap and needs to be multiple-annotated
    new_tilepointer_x = tilepointer_x
    new_tilepointer_y = tilepointer_y
    # if the rightmost x tile-coordinate of the box is in the left overlap zone, also put it in the left-adjacent box
    if intile_rx < tile_overlap + disregard_threshold*box_dimensionx:
        new_tilepointer_x = tilepointer_x-1
        #print("left margin")
    # or if the leftmost x tile-coordinate of the box is in the right overlap zone, also put it in the right-adjacent box
    elif intile_lx > tile_width - (tile_overlap + disregard_threshold*box_dimensionx):
        new_tilepointer_x = tilepointer_x+1
        #print("right margin")
    # if the bottommost y tile-coordinate is in the top overlap zone, also put it in the top-adjacent box
    if intile_by < tile_overlap + disregard_threshold*box_dimensiony:
        new_tilepointer_y = tilepointer_y-1
        #print("top margin")
    # if the topmost y tile-coordinate is in the bottom overlap zone, also put it in the bottom-adjacent box
    elif intile_ty > tile_height - (tile_overlap+disregard_threshold*box_dimensiony):
        new_tilepointer_y = tilepointer_y+1
        #print("bottom margin")
    # if the left remainder has changed, change the tile pointer
    if new_tilepointer_x != tilepointer_x:
        tile_pointer = "[{x}, {y}]".format(x=int(new_tilepointer_x*x_tile_dist), y=int(tilepointer_y*y_tile_dist))
        try:
            tile_info = inv_map[tile_pointer]
            entry.append(tile_info)
        except:
            print("adjacent tile does not exist")
    # if the top remainder has changed, change the tile pointer
    if new_tilepointer_y != tilepointer_y:
        tile_pointer = "[{x}, {y}]".format(x=int(tilepointer_x*x_tile_dist), y=int(new_tilepointer_y*y_tile_dist))
        try:
            tile_info = inv_map[tile_pointer]
            entry.append(tile_info)
        except:
            print("adjacent tile does not exist")
    # if both have changed, change tile pointer to bishop (diagonal) tile
    if new_tilepointer_x != tilepointer_x and new_tilepointer_y != tilepointer_y:
        #print("double margin")
        tile_pointer = "[{x}, {y}]".format(x=int(new_tilepointer_x*x_tile_dist), y=int(new_tilepointer_y*y_tile_dist))
        try:
            tile_info = inv_map[tile_pointer]
            entry.append(tile_info)
        except:
            print("bishop tile does not exist")
    return entry

In [10]:
def SortFunc(e):
  return e['tile_ID']

# this function converts orthomosaic coordinates to tile coordinates
def orthoboxes_to_tileboxes(box_list, tiling_scheme):
    tile_entry_list = []
    img_data = tiling_scheme["tile_pointers"] # top-left pixel location for each image, in orthomosaic coordinates
    for orthobox in orthoboxes:
        tile_ID = assign_tiles(orthobox['box'], tiling_scheme)
        for x in tile_ID:
            x_offset, y_offset = img_data['image_locations'][x]
            tilebox = []
            for point in orthobox['box']:
                new_x = point[0]-x_offset
                new_y = point[1]-y_offset
                if new_x > tiling_scheme["tile_width"]:
                    new_x = tiling_scheme["tile_width"]
                elif new_x < 0:
                    new_x = 0
                if new_y > tiling_scheme["tile_height"]:
                    new_y = tiling_scheme["tile_height"]
                elif new_y < 0:
                    new_y = 0
                tilebox.append((new_x, new_y))
            # reassemble boxes entries with tile name and coordinates
            try:
                new_entry = {"tile_ID":x, "box": tilebox, "class": orthobox['class']} ####### make sure multiple entries possible per old entry, and take care of class!
                tile_entry_list.append(new_entry)
            except:
                print("entry was blank")
    from natsort import natsorted
    tile_entry_list = natsorted(tile_entry_list, key=SortFunc)
    return tile_entry_list
# 
tileboxes = orthoboxes_to_tileboxes(orthoboxes, tiling_scheme)
pprint(tileboxes[0:5])

[{'box': [(682.0408048033714, 598.3117606882006),
          (800, 598.3117606882006),
          (800, 337.3939670854015),
          (682.0408048033714, 337.3939670854015)],
  'class': 'Bottlebrush unk.',
  'tile_ID': 'FCAT2APPK---69.png'},
 {'box': [(195.04080480337143, 598.3117606882006),
          (457.7117680609226, 598.3117606882006),
          (457.7117680609226, 337.3939670854015),
          (195.04080480337143, 337.3939670854015)],
  'class': 'Bottlebrush unk.',
  'tile_ID': 'FCAT2APPK---70.png'},
 {'box': [(324.53042992949486, 688.8260060920147),
          (587.201393187046, 688.8260060920147),
          (587.201393187046, 427.90821248921566),
          (324.53042992949486, 427.90821248921566)],
  'class': 'Bottlebrush unk.',
  'tile_ID': 'FCAT2APPK---70.png'},
 {'box': [(446.02713003754616, 715.8214830886573),
          (708.6980932950974, 715.8214830886573),
          (708.6980932950974, 454.9036894858582),
          (446.02713003754616, 454.9036894858582)],
  'class': 'Bottl

In [11]:
### This section is only helpful if you are using VIA annotations.
### If your system requires a different formate (e.g. Coco, Yolo)
### you should code your own function to format that output, see:
### https://towardsdatascience.com/image-data-labelling-and-annotation-everything-you-need-to-know-86ede6c684b1
### https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc

import os, csv
def tileboxes_to_VIA_file(tileboxes, tiling_scheme_file, output_dir):
    via_array = [["filename",
                 "file_size",
                 "file_attributes",
                 "region_count",
                 "region_id",
                 "region_shape_attributes",
                 "region_attributes"]]
    img_data = tiling_scheme["tile_pointers"]
    remnant_tiles = list(img_data["image_locations"].keys())

    filename = ""
    for tilebox in tileboxes:
        temp = []
        if filename != tilebox["tile_ID"]:
            filename = tilebox["tile_ID"]
            remnant_tiles.remove(filename)
            count = 0
        else:
            count += 1
        file_size = ""
        file_attributes = "{}"
        x_coordinates, y_coordinates = zip(*tilebox["box"])
        x1 = int(min(x_coordinates))
        y1 = int(min(y_coordinates))
        x2 = int(max(x_coordinates))
        y2 = int(max(y_coordinates))

        region_shape_attributes = {"name":"rect", "x":x1, "y":y1, "width":x2-x1, "height":y2-y1}
        region_count = ""
        region_attributes = {class_colname:tilebox["class"]}
        region_ID = count
        via_array.append([filename, file_size, file_attributes, region_count, region_ID, region_shape_attributes, region_attributes])

    for k, x in enumerate(via_array):
        via_array[k][5],via_array[k][6] = str(x[5]).replace("'",'"'),str(x[6]).replace("'",'"')

    for k in remnant_tiles:
        via_array.append([k, '', '{}', '', '', '', '' ])

    # Set output directory, create it if necessary
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    input_dir = tiling_scheme_file.split("/")[-2]

    # write out new VIA file with additional detections
    with open(output_dir + "/{i}_VIA_annotations_{tw}_{ov}.csv".format(i=input_dir, tw=tiling_scheme["tile_width"], ov=tiling_scheme["tile_overlap"]), 'w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerows(via_array)

    print("Annotations rewritten to " + output_dir + "/{i}_VIA_annotations_{tw}_{ov}.csv".format(i=input_dir, tw=tiling_scheme["tile_width"], ov=tiling_scheme["tile_overlap"]))
    
tileboxes_to_VIA_file(tileboxes, tiling_scheme_file, output_dir)

### The resulting file can be opened in VIA: https://www.robots.ox.ac.uk/~vgg/software/via/
### Download the application and open 'via.html'
### Import the CSV file: Annotation > Import Annotations (from CSV)
### Add image files: Project > Add local files
### Use interface to review and clean annotations, as desired

Annotations rewritten to D:/FCAT/annotations/tiles2_VIA_annotations_800_313.csv
